In [ ]:
pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 30.5 MB/s 
     |████████████████████████████████| 41 kB 742 kB/s 


In [ ]:
pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.7 MB/s 
     |████████████████████████████████| 5.5 MB 52.1 MB/s 
     |████████████████████████████████| 1.3 MB 59.8 MB/s 
     |████████████████████████████████| 163 kB 60.1 MB/s 
     |████████████████████████████████| 7.6 MB 59.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=64081155e09c13376c88144a2f458f15e35f7077cfa4f43c59389d4c5a3c9050
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, OWL, RDF, RDFS, SKOS, VOID, XMLNS, XSD
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.extras import describer
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
g= Graph()
g.bind("owl",OWL)
g.bind("pr","http://example.org/NLPOntology/")
ns_url = "http://example.org/NLPOntology/"
g.add((URIRef('http://example.org/NLPOntology/'), RDF.type, OWL.Ontology ))

<Graph identifier=N311d58f2cd6b427e854f8121658dde8a (<class 'rdflib.graph.Graph'>)>

In [ ]:
df = pd.read_json('/content/FDF_for_amine.json')
df2 = pd.read_json('/content/Clean_Database.json')

In [ ]:
df_process=df2.drop_duplicates(subset='Title_2', keep='first', inplace=False)
df_process = df_process['Title_2']

In [ ]:
for c in df_process:
  cl = URIRef(ns_url+c.replace(" ","_"))
  g.add((cl, RDF.type, OWL.Class))
  clOutput =URIRef(ns_url+c.replace(" ","_")+"_"+"Outputs".replace(" ","_"))
  g.add((clOutput, RDF.type, OWL.Class))
  clTools =URIRef(ns_url+c.replace(" ","_")+"_"+"Tools and techniques".replace(" ","_"))
  g.add((clTools, RDF.type,OWL.Class))
  clInput =URIRef(ns_url+c.replace(" ","_")+"_"+"Inputs".replace(" ","_"))
  g.add((clInput, RDF.type, OWL.Class))

In [ ]:
for i in range (216):
  if df['Relation_type'][i]=="Inputs":
     df['Relation_type'][i]="Has_Inputs"
  if df['Relation_type'][i]=="Outputs":
     df['Relation_type'][i]="Has_Outputs"
  if df['Relation_type'][i]=="Tools and techniques":
     df['Relation_type'][i]="Has_Tools_and_techniques"

In [ ]:
df.rename(columns={"Relation_type":"Type"}, inplace=True)

In [ ]:
df.rename(columns={"Title":"Concept"}, inplace=True)

In [ ]:
for i in range(len(df['Similarity'])):
  for j in range(len(df['Similarity'][i])):
    df['Similarity'][i][j] = df['Similarity'][i][j][0]

In [ ]:
for i in range(len(df['Similarity'])):
  df['Similarity'][i] = df['Similarity'][i] + df['Synonyms'][i]

In [ ]:
df = df.drop(columns=['Synonyms'])

In [ ]:
df.rename(columns={"Similarity":"Synonyms"}, inplace=True)

In [ ]:
for i in range(len(df['Synonyms'])):
  df['Synonyms'][i] = set(df["Synonyms"][i])
  df['Synonyms'][i] = list(df["Synonyms"][i])

In [ ]:
df.head(1)

,Section,Concept,Type,Corpus,Synonyms,Keywords,Entities,Ref,Definition,Concept_range,Relation_Non_Taxonomique
0,1.0.0.0,Project scope management,,Project Scope Management includes the process ...,[scope management plan Document project produc...,"[[scope, 0.48860000000000003], [management, 0....","[[[Project, Scope, Management]], [[process]], ...","[[[[['Figure 5-1'], []], []], []], []]",Project Scope Management includes the process ...,"[Project Scope Management, defining, Collect R...","[includes, required ensure, includes, required..."


Add Concepts and Processes

In [ ]:
for o in df.index :
    for c in df_process: 
        if c == df.loc[o,'Concept'] and  df.loc[o,'Type'] == 'Has_Outputs' :
        
            clo = URIRef(ns_url+c.replace(" ","_")+"_"+"Outputs".replace(" ","_"))
            ind = URIRef(ns_url+df.loc[o,'Concept'].replace(" ","_"))
            g.add((ind, RDF.type, clo))
        

        if  c == df.loc[o,'Concept'] and df.loc[o,'Type'] == 'Has_Inputs' :


            clo = URIRef(ns_url+c.replace(" ","_")+"_"+"Inputs".replace(" ","_"))
            ind = URIRef(ns_url+df.loc[o,'Concept'].replace(" ","_"))
            g.add((ind, RDF.type, clo))
     
        if c == df.loc[o,'Concept'] and df.loc[o,'Type'] == 'Has_Tools_and_techniques' :


            clo = URIRef(ns_url+c.replace(" ","_")+"_"+"Tools and techniques".replace(" ","_"))
            ind = URIRef(ns_url+df.loc[o,'Concept'].replace(" ","_"))
            g.add((ind, RDF.type, clo))

Object Property

In [ ]:
for i in df.index :
  if df.loc[i,'Type'] !='':

    if df.loc[i,'Type'] == 'Has_Tools_and_techniques' :
        
       c = URIRef(ns_url+df.loc[i,'Type'].replace(" ","_"))
       domaine = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_"))
       rang = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_")+"_Tools and techniques".replace(" ","_"))
    elif df.loc[i,'Type'] == 'Has_Inputs' :
        
      c = URIRef(ns_url+df.loc[i,'Type'].replace(" ","_"))
      domaine = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_"))
      rang = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_")+"_Inputs".replace(" ","_"))
    
    elif df.loc[i,'Type'] == 'Has_Outputs' :
        
      c = URIRef(ns_url+df.loc[i,'Type'].replace(" ","_"))
      domaine = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_"))
      rang = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_")+"_Outputs".replace(" ","_"))
    
    g.add((c, RDF.type, OWL.ObjectProperty))
    g.add((c, RDFS.domain, domaine))
    g.add((c, RDFS.range, rang))

Add definition (Annotation DefinedBy)

In [ ]:
for i in range(len(df)) :
    c = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_"))
    desc=df.loc[i,'Definition']
    definedby = Literal(desc,datatype=XSD.string)
    g.add((c, RDFS.isDefinedBy, definedby))

Adding Synonyms

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
for i in range(len(df)):    
    for j in range(len(df['Synonyms'][i])):
      syn = Literal( df['Synonyms'][i][j] ,datatype=XSD.string)
      c = URIRef(ns_url+df['Synonyms'][i][j].replace(" ","_"))
    g.add((c, RDFS.seeAlso, syn))

In [ ]:
for i in range(len(df)):
    c = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_"))
    for j in range(len(df.loc[i,'Concept'])):
        syn = Literal( df['Synonyms'][j][0] ,datatype=XSD.string)
        g.add((c, RDFS.seeAlso, syn))

Data Property

In [ ]:
df_concept_range = pd.read_json('/content/concept_range_for_ontology.json')

In [ ]:
for i in range(len(df_concept_range)):
    c = URIRef(ns_url+df_concept_range.loc[i,'Rlation_taxonomique'].replace(" ","_"))
    domaine = URIRef(ns_url+df_concept_range.loc[i,'Entity'].replace(" ","_"))
    c_range = Literal(df_concept_range.loc[i,'Concept_Range'],datatype=XSD.string)
    g.add((c, RDF.type, OWL.DatatypeProperty))
    g.add((c, RDFS.domain, domaine))
    g.add((c, RDFS.range, XSD.string))
    g.add((domaine,c,c_range))

In [ ]:
#for i in range(len(df_concept_range)):
 #   indiv = URIRef(ns_url+df_concept_range.loc[i,'Entity'].replace(" ","_"))
  #  section = Literal(df_concept_range.loc[i,'Concept_Range'],datatype=XSD.string)
   # data_prop=URIRef(ns_url+df_concept_range.loc[i,'Rlation_taxonomique'].replace(" ","_"))
#
 #   g.add((indiv,data_prop,section))

Add Reference

In [ ]:
df_figures = pd.read_json("/content/fdf_for_marouene.json")

In [ ]:
df_figures2 = pd.read_json("/content/FFDF_for_amine.json")

Add Sections:

In [ ]:
list_ref = []
for j in range(len(df)):
  list_ref.append(df_figures['Ref'][j][0][0][1])
print(list_ref)

['[]', "['Section 2.3', 'Section 2.2', 'Section 4.2.3.1']", "['Section 4.1.3.1']", "['Section 8.1.3.1', 'Section 4.2.3.1']", '[]', '[]', "['Section 4.1.2.1']", '[]', '[]', '[]', '[]', '[]', "['Section 4.1.3.1']", "['Section 5.1.3.1', 'Section 5.1.3.2', 'Section 4.2.3.1']", "['Section 4.1.3.2', 'Section 4.4.3.1']", '[]', '[]', '[]', '[]', "['Section 4.1.2.1']", "['Section 8.1.2.2', 'Section 4.1.2.2']", "['Section 4.5.2.2']", '[]', '[]', "['Section 4.1.2.3']", '[]', '[]', '[]', '[]', '[]', "['Section 4.1.3.1']", "['Section 5.1.3.1', 'Section 4.2.3.1']", "['Section 4.1.3.2']", '[]', '[]', "['Section 4.1.2.1']", '[]', "['Section 5.1.2.2', 'Section 8.1.2.4']", "['Section 4.1.2.3']", '[]', '[]', "['Section 5.2.3.1', 'Section 4.1.3.2', 'Section 5.2.3.2']", '[]', "['Section 5.1.3.1']", "['Section 5.2.3.1', 'Section 5.3.3.1']", '[]', '[]', "['Section 4.1.2.1']", '[]', "['Section 5.3.3.1']", "['Section 5.2.3.1', 'Section 4.1.3.2']", '[]', "['Section 5.1.3.1', 'Section 5.1.3.2', 'Section 4.2.3.1'

In [ ]:
for i in range(len(df)):
    c = URIRef(ns_url+'DescribedInSection')
    concept = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_"))
    section = Literal(list_ref[i],datatype=XSD.string)
    g.add((c, RDF.type, OWL.DatatypeProperty))
    g.add((c, RDFS.domain, concept))
    g.add((c, RDFS.range, XSD.string))
    g.add((domaine,c,section))

Add Figures:

In [ ]:
list_fig = []
list_fig2 = []
fig=''
for j in range(len(df)):
  if len(df_figures2['Ref'][j][0][0][0][0])!=0:
    for x in range(len(df_figures2['Ref'][j][0][0][0][0])):
      list_fig2.append(df_figures2['Ref'][j][0][0][0][0][x][3] +'Link :'+df_figures2['Ref'][j][0][0][0][0][x][0] )
    list_fig.append('#'.join(fig for fig in list_fig2))
    list_fig2 = []
  else : list_fig.append('No_figure')

In [ ]:
for i in range(len(df)):
    c = URIRef(ns_url+'ShownInFigure')
    concept = URIRef(ns_url+df.loc[i,'Concept'].replace(" ","_"))
    figure = Literal(list_fig[i],datatype=XSD.string)
    g.add((c, RDF.type, OWL.DatatypeProperty))
    g.add((c, RDFS.domain, concept))
    g.add((c, RDFS.range, XSD.string))
    g.add((domaine,c,figure))

In [ ]:
g.serialize(destination='ontologyfinal.owl', format='turtle')

<Graph identifier=N311d58f2cd6b427e854f8121658dde8a (<class 'rdflib.graph.Graph'>)>

Evaluation

In [ ]:
!pip install rouge
from rouge import Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
all_data_combined =  df['Concept'].tolist() + df['Definition'].tolist()

all_data_combined2 =  df['Definition'].tolist()+ df['Concept'].tolist()+ df['Relation_Non_Taxonomique'].tolist()+ df['Concept_range'].tolist() 

all_data_combined3 =  df['Definition'].tolist()+ df['Concept'].tolist()+ df['Relation_Non_Taxonomique'].tolist()+ df['Concept_range'].tolist() +df['Ref'].tolist() + df['Synonyms'].tolist()


In [ ]:
ch=''
for e in all_data_combined:
    ch+='. '+str(e)
ch2=''
for e in all_data_combined2:
    ch2+='. '+str(e)

ch3=''
for e in all_data_combined3:
    ch3+='. '+str(e)

In [ ]:
df_unclean = pd.read_json('DataBase.json')

In [ ]:
proc = ','.join(list(df_unclean['Corpus'].values))

In [ ]:
proc

' Project Scope Management includes the processes required to ensure that the project includes all the work required,  and only the work required, to complete the project successfully. Managing the project scope is primarily concerned with  deﬁning and controlling what is and is not included in the project. The Project Scope Management processes are: 5.1 Plan Scope Management—The process of creating a scope management plan that documents how the project  and product scope will be deﬁned, validated, and controlled. 5.2 Collect Requirements—The process of determining, documenting, and managing stakeholder needs and  requirements to meet project objectives. 5.3 Define Scope—The process of developing a detailed description of the project and product. 5.4 Create WBS—The process of subdividing project deliverables and project work into smaller, more manageable  components. 5.5 Validate Scope—The process of formalizing acceptance of the completed project deliverables. 5.6 Control Scope—The pr

In [ ]:
import sys
sys.setrecursionlimit(1500)

Concept+Definition

In [ ]:
rouge = Rouge()
score=rouge.get_scores(proc,ch , avg=True)
print('Precision = '+str(score['rouge-1']['p']))
print('Recall = ' +str(score['rouge-1']['r']))
print('f-measure = '+str(score['rouge-1']['f']))

KeyboardInterrupt: ignored

Concept+Definition+Concept_range+Relation_tax

In [ ]:
score2=rouge.get_scores(proc,ch2 , avg=True)
print('Precision = '+str(score2['rouge-1']['p']))
print('Recall = ' +str(score2['rouge-1']['r']))
print('f-measure = '+str(score2['rouge-1']['f']))

Concept+Definition+Concept_range+Relation_tax+Ref

In [ ]:
rouge = Rouge()
score3=rouge.get_scores(proc,ch3 , avg=True)
print('Precision = '+str(score3['rouge-1']['p']))
print('Recall = ' +str(score3['rouge-1']['r']))
print('f-measure = '+str(score3['rouge-1']['f']))

In [ ]:
df.to_json('final_df_for_recommendation.json')

In [ ]:
df.to_csv('final_df_for_recommendation.csv')

In [ ]:
g.all_nodes()